In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('TRACE2014_jinming_5000.csv.xlsx')

In [3]:
data.head(3)

,TRC_ST,BOND_SYM_ID,CUSIP_ID,SCRTY_TYPE_CD,WIS_CD,CMSN_TRD_FL,ENTRD_VOL_QT,RPTD_PR,YLD_SIGN_CD,YLD_PT,...,Affiliated,Offer_d_Diff,Maturity_d_Diff,Offer_d30,Offer_d60,Offer_d90,Mature_d90,YYYYQQ,Buyer_RPT,Seller_RPT
0,T,TWIX3666829,982526AB1,CORP,N,C,25000,102.992000,NaN,1.571479,...,0,3292,364.0,1,1.0,1.0,NaN,2014-09-30,10787.0,NaN
1,T,WAMU3885939,92936PAB6,CORP,N,C,214,95.934579,NaN,NaN,...,0,1009,5565.0,1,1.0,1.0,NaN,2014-12-31,NaN,NaN
2,T,WAMU3885939,92936PAB6,CORP,N,C,100,92.500000,NaN,NaN,...,0,991,5583.0,1,1.0,1.0,NaN,2014-12-31,NaN,10796.0


In [4]:
#document = data.groupby(by=['Report_Dealer_Index','TRD_EXCTN_DT'])
document = data[['Report_Dealer_Index','TRD_EXCTN_DT']]
document.head(3)

,Report_Dealer_Index,TRD_EXCTN_DT
0,0,20140716
1,83,20141223
2,83,20141205


In [5]:
token = data[['BOND_SYM_ID','CUSIP_ID']]
token.head(3)

,BOND_SYM_ID,CUSIP_ID
0,TWIX3666829,982526AB1
1,WAMU3885939,92936PAB6
2,WAMU3885939,92936PAB6


In [6]:
data_gb = data.groupby(by=['Report_Dealer_Index','TRD_EXCTN_DT','BOND_SYM_ID'])

In [7]:
matrix_1 = data_gb.size()
matrix_1.head()

Report_Dealer_Index  TRD_EXCTN_DT  BOND_SYM_ID
0                    20140102      AA.HO          1
                                   BCS.KBJ        1
                                   MT3824014      1
                                   PBR3674838     1
                                   PBR4006643     1
dtype: int64

In [8]:
matrix_2 = data_gb['ENTRD_VOL_QT'].sum()
matrix_2.head()

Report_Dealer_Index  TRD_EXCTN_DT  BOND_SYM_ID
0                    20140102      AA.HO            7000
                                   BCS.KBJ        150000
                                   MT3824014       10000
                                   PBR3674838      20000
                                   PBR4006643      30000
Name: ENTRD_VOL_QT, dtype: int64

In [9]:
matrix_3 = data_gb[['ENTRD_VOL_QT','RPTD_PR']]
matrix_3.prod()

ENTRD_VOL_QT       RPTD_PR
Report_Dealer_Index TRD_EXCTN_DT BOND_SYM_ID                            
0                   20140102     AA.HO        7.000000e+03     92.533000
                                 BCS.KBJ      1.500000e+05    106.440000
                                 MT3824014    1.000000e+04    106.750000
                                 PBR3674838   2.000000e+04    112.876900
                                 PBR4006643   3.000000e+04     89.160000
                    20140106     BBY.GH       1.000000e+05    103.170000
                                 TEF3675700   3.000000e+04    102.811000
                                 X3984308     1.780000e+05    105.250000
                    20140107     DELL.GR      2.500000e+04     89.500000
                                 ECOH4050412  5.000000e+04    104.900000
                                 ERJ3866394   5.000000e+04    100.700000
                                 LYG3688423   2.500000e+04    106.526000
                                 MT3824016    5.000000e+04    109.150000
                                 PBR3704563   2.000000e+04    115.776000
                                 WRB.AD       5.000000e+04    111.768000
                    20140108     AU3882696    5.000000e+04     86.815000
                                 BBY4029053   2.500000e+04    104.000000
                                 LYG3688423   3.000000e+04    106.418000
                                 MT.AK        1.000000e+04    102.625000
                                 VALE3701084  1.000000e+05     95.612000
                    20140109     AKS.GH       2.600000e+04     98.750000
                                 ANKO.GC      5.000000e+03    102.805000
                                 GOL.GC       1.000000e+05     96.000000
                                 GS.AOK       1.000000e+05    109.300000
                                 HBC.TF       1.000000e+05     99.411000
                                 IBM.AA       5.000000e+04     96.041000
                                 JCP.HA       2.500000e+04     76.055000
                                 JNC3667148   1.800000e+05    100.420000
                                 MT.AJ        1.000000e+05    106.000000
                                 TAM.GB       1.000000e+05    104.600000
...                                                    ...           ...
120                 20141008     CTL3697232   4.500000e+04    117.249000
                                 DRI3911609   1.200000e+09   9116.621361
                                 FCX4060859   1.200000e+09  10258.651225
                                 PBR4006643   5.000000e+04     96.918000
                                 PSEC3980466  5.000000e+04    107.567000
                                 RIG.HG       7.500000e+08  11578.836025
                                 URS4080008   1.000000e+09  10878.490000
                    20141014     ANR.GB       1.100000e+05     47.145000
                    20141022     CAS3867594   5.000000e+04    104.355000
                    20141031     NEM.GL       1.200000e+04     99.671000
                                 PBR4106372   1.200000e+04    108.426000
                                 RIG.GZ       4.000000e+08  11723.692176
                                 RIG.HE       1.200000e+04    105.300000
                                 SWY.GH       1.000000e+04    106.187000
                    20141112     AAPL4122392  2.500000e+04    103.290000
                                 CBS3825072   3.000000e+04    102.351000
                                 COF4118914   2.500000e+04    100.894000
                                 HUM4164825   2.500000e+04    101.138000
                                 MAR4172091   3.000000e+04    101.997000
                                 PM3699034    2.500000e+04    102.054000
                    20141114     KMI4035394   1.500000e+04    101.031000
                                 MUR3856177   1.000000e+04     99.128000
                    20141118   

### Validation

In [10]:
shape = data.shape
print('{} rows {} columns'.format(shape[0],shape[1]))

5000 rows 57 columns


In [11]:
print('{} rows that are entirely the same in the data set'.format(data.duplicated().sum()))

0 rows that are entirely the same in the data set


In [12]:
print('Number of duplcations based on grouping keys:')
test_duplication = [['BOND_SYM_ID'],['CUSIP_ID'],['BOND_SYM_ID','CUSIP_ID'],['Report_Dealer_Index','TRD_EXCTN_DT'],
                   ['Report_Dealer_Index','TRD_EXCTN_DT','BOND_SYM_ID']]
for test in test_duplication:
    print('{} : {}'.format(test, data.duplicated(subset=test).sum()))

Number of duplcations based on grouping keys:
['BOND_SYM_ID'] : 4087
['CUSIP_ID'] : 4095
['BOND_SYM_ID', 'CUSIP_ID'] : 4087
['Report_Dealer_Index', 'TRD_EXCTN_DT'] : 4154
['Report_Dealer_Index', 'TRD_EXCTN_DT', 'BOND_SYM_ID'] : 1104


In [13]:
data.loc[data['BOND_SYM_ID']=='TWC3675093']

,TRC_ST,BOND_SYM_ID,CUSIP_ID,SCRTY_TYPE_CD,WIS_CD,CMSN_TRD_FL,ENTRD_VOL_QT,RPTD_PR,YLD_SIGN_CD,YLD_PT,...,Affiliated,Offer_d_Diff,Maturity_d_Diff,Offer_d30,Offer_d60,Offer_d90,Mature_d90,YYYYQQ,Buyer_RPT,Seller_RPT
3659,T,TWC3675093,88732JAS7,CORP,N,NaN,25000,125.450,NaN,2.854074,...,0,1794,1867.0,1,1.0,1.0,NaN,2014-03-31,60656.0,NaN
4999,T,TWC3675093,88732JAS7,CORP,N,NaN,50000,126.169,NaN,2.390999,...,0,1920,1741.0,1,1.0,1.0,NaN,2014-06-30,81572.0,NaN
